In [597]:
import os
import pandas as pd
import csv
import re

In [572]:
INPUT_DIR = '../input_data/IFs'
OUTPUT_DIR = '../input_data/IFs_cleaned'

In [573]:
files = [
    f"{INPUT_DIR}/{f}" for f in os.listdir(INPUT_DIR)
    if os.path.isfile(os.path.join(INPUT_DIR, f))
]

In [ ]:
def get_ifs_name(source):
    return re.sub(r'^\d+\. ', '', files[0].replace(f"{INPUT_DIR}/", "")).replace(".csv", "")

In [574]:
def cleanup_semicolon(source):
    with open(files[0], 'r') as file:
        content = file.read()
    updated_content = content.replace(';', '')
    with open(files[0], 'w') as file:
        file.write(updated_content)

In [575]:
cleanup_semicolon(files[0])

In [576]:
data = pd.read_csv(files[0], header=[1,4,5], sep=',')

In [577]:
new_columns = list(data.columns)
for i, col in enumerate(new_columns):
    if col == ('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2'):
        new_columns[i] = 'Year'

In [578]:
data.columns = new_columns

In [579]:
df = pd.DataFrame(data.to_dict('records'))

In [580]:
df_melted = df.melt(id_vars=['Year'], var_name='variable', value_name='value')

In [581]:
new_data = []
for value_list in df_melted.to_dict('records'):
    new_data.append({
        "year": value_list["Year"],
        "country": value_list["variable"][0],
        "unit": value_list["variable"][1],
        "value_type": list(filter(lambda v:v,value_list["variable"][2].split("_"))),
        "value": value_list["value"]
    })

In [582]:
df = pd.DataFrame(new_data)

In [583]:
df_split = pd.DataFrame(df['value_type'].tolist(), index=df.index)
df_split.columns = ['value_name', 'jmp_category', 'commitment','other']
df_final = pd.concat([df, df_split], axis=1)
df_final = df_final[['year','country','unit','value_name','jmp_category','commitment','value']]

In [594]:
df_final.to_csv("testing.csv", index=False)